In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('../2_data_preparation/cleaned_data.csv')
clustered = pd.read_csv('data_clustered.csv')

In [3]:
clustered = clustered[['name', 'cluster']]
clustered

,name,cluster
0,Lerone Murphy,4
1,Edson Barboza,4
2,Khaos Williams,4
3,Carlston Harris,2
4,Themba Gorimbo,2
...,...,...
2391,Nick Sanzo,3
2392,Jim Mullen,3
2393,Yoshiki Takahashi,1
2394,Wallid Ismail,1


In [4]:
data['winner'].value_counts()

winner
f1    7289
0      111
Name: count, dtype: int64

In [5]:
# filter data where clear decision
data = data[data['winner'] == 'f1']
data['winner'].value_counts()

winner
f1    7289
Name: count, dtype: int64

In [6]:
odds_df = data[['f1', 'f2']]

In [7]:
odds_df.head(2)

,f1,f2
0,Lerone Murphy,Edson Barboza
1,Khaos Williams,Carlston Harris


In [8]:

# Merge the cluster column from clustered DataFrame based on fighter names (f1 and f2)
odds_df = pd.merge(odds_df, clustered[['name', 'cluster']], left_on='f1', right_on='name', how='left')
odds_df.rename(columns={'cluster': 'winner_cluster'}, inplace=True)

odds_df = pd.merge(odds_df, clustered[['name', 'cluster']], left_on='f2', right_on='name', how='left')
odds_df.rename(columns={'cluster': 'loser_cluster'}, inplace=True)

In [9]:
odds_df.head(2)

,f1,f2,name_x,winner_cluster,name_y,loser_cluster
0,Lerone Murphy,Edson Barboza,Lerone Murphy,4,Edson Barboza,4
1,Khaos Williams,Carlston Harris,Khaos Williams,4,Carlston Harris,2


In [10]:
odds_df.drop(columns=['f1', 'f2', 'name_x', 'name_y'], inplace=True)

In [11]:
odds_df['winner_cluster'].value_counts()

winner_cluster
2    3574
4    2981
1     643
0      65
3      26
Name: count, dtype: int64

In [12]:
import matplotlib.cm as cm

# Group by winner and loser clusters to count the number of wins and losses
win_counts = odds_df.groupby(['winner_cluster', 'loser_cluster']).size().unstack(fill_value=0)
loss_counts = odds_df.groupby(['loser_cluster', 'winner_cluster']).size().unstack(fill_value=0)

# Calculate win percentage by cluster matchup
win_percentage = (win_counts / (win_counts + loss_counts)) * 100
# Round win percentages to nearest integer
win_percentage_rounded = win_percentage.round()

# Convert rounded win percentages to integers
# win_percentage_int = win_percentage_rounded.astype(int)

# Display win percentages by cluster matchup
win_percentage_rounded.style.background_gradient(cmap='Blues', axis=None)

loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,50.000000,36.000000,45.000000,100.000000,37.000000
1,64.000000,50.000000,28.000000,87.000000,26.000000
2,55.000000,72.000000,50.000000,93.000000,48.000000
3,0.000000,13.000000,7.000000,50.000000,0.000000
4,63.000000,74.000000,52.000000,100.000000,50.000000


In [13]:
(win_counts + loss_counts)

loser_cluster,0,1,2,3,4
winner_cluster,,,,,
0,2,25,94,2,30
1,25,296,1057,15,684
2,94,1057,3520,54,1984
3,2,15,54,40,5
4,30,684,1984,5,2820
